<h1>Actividad Clasificador de topicos</h1>
<h2>Martes 10 de Agosto</h2>
<ol>
<li>Elegir dos topicos</li>
<li>Obtener al menos 50 frases para cada topico</li>
<li>Definir un set con almenos 25 frases de cada tópico</li>
<li>Definir el código para entrenar un modelo de SGD o Naive Bayes</li>
</ol>



In [42]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_array
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import defaultdict
import csv, sys

In [21]:
canciones_df=pd.read_csv('data/Canciones Romanticas.csv')
documentacion_df=pd.read_csv('data/Lenguajes de programacion.csv')
canciones_df=canciones_df.iloc[:, 0].tolist()
documentacion_df=documentacion_df.iloc[:, 0].tolist()

In [22]:
canciones_df = list(map(str.lower,canciones_df))
documentacion_df = list(map(str.lower,documentacion_df))

In [23]:
canciones_df

["i've got sunshine on a cloudy day with my girl",
 "i've got a sweeter song than the birds in the trees",
 'give you some time to prove that i can trust you again',
 'and catch the tears whenever you cry',
 "and in my dreams i've kissed your lips a thousand times",
 "i'm out of my head hopelessly devoted to you",
 'and now you know i can´t smile without you',
 'i go out every night and sleep all day',
 "just to let you know that i'm still around",
 'well life would still go on believe me',
 'oh then turn around and make up',
 "you're all i've ever wanted and my arms are open wide",
 "if she said that's the way it ought to be",
 'though life would still go on believe me',
 "now my baby's dancin' but she's dancin' with another man",
 "and that you'd be reminded that for me, it isn't over",
 "i'll think of you each step of the way",
 'somewhere in her smile she knows',
 "it's been so lonely without you here",
 "you're my end and my beginning",
 'is there really no chance to start once ag

In [24]:
categories = ['canciones', 'documentacion']

In [25]:
# Join individual documents to calculate cosine similarity
canciones = ' '.join(canciones_df)
documentacion = ' '.join(documentacion_df)
documents = [canciones,documentacion]
count_vectorizer = CountVectorizer(stop_words="english")
#count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vectorizer.get_feature_names_out(),
   index=["canciones", "documentacion"],
)
print(df)
print(cosine_similarity(df, df))

               abstract  accept  allows  arguments  arms  assigned  \
canciones             0       0       0          0     1         0   
documentacion         1       2       6          2     0         1   

               assignment  async  asynchronously  attempts  ...  way  weapon  \
canciones               0      0               0         0  ...    3       1   
documentacion           1      2               1         1  ...    1       0   

               weeping  wide  willing  willow  wise  word  wrapped  write  
canciones            1     1        1       1     1     1        1      0  
documentacion        0     0        0       0     0     0        0      1  

[2 rows x 287 columns]
[[1.        0.0338322]
 [0.0338322 1.       ]]


In [26]:
df.shape

(2, 287)

Identify common tokens and resolve

the first approach, delete all, is not right; delete those elements that are less frequent. Example: paper [105 0 5] delete for segment 15 and 32, if the number is the same, delete the word is all bags of words.

There are 201 terms.

Create three lists to keep track of which tokens will be erased for each segment

In [29]:
i = 0
commonTerms = []
cancionesdelete = []
documentaciondelete = []
for (columnName, columnData) in df.items():
    nOcurrences = sum(x != 0 for x in columnData.values)
    if nOcurrences > 1:
      i = i + 1
      commonTerms.append(columnName) # keep track of all the common terms
      print('Token : ', columnName)
      print('Token Frequency : ', columnData.values)
      print('Token Repetition', sum(x != 0 for x in columnData.values))
      cancionescount = columnData.values[0]
      documentacioncount = columnData.values[1]
      print('Canciones Token count ', cancionescount)
      print('Documentacion Token count ', documentacioncount)
      # Less frequent tokens are deleted from the bag of words
      if (cancionescount > documentacioncount) and (cancionescount > documentacioncount):
        print('Canciones wins')
        documentaciondelete.append(columnName)
      elif (documentacioncount > cancionescount) and (documentacioncount > cancionescount):
        print('Documentacion wins')
        cancionesdelete.append(columnName)
      else: # there is a tie, the tokens is destroyed
        cancionesdelete.append(columnName)
        documentaciondelete.append(columnName)
print('Total terms ', i)

Token :  catch
Token Frequency :  [1 2]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  2
Documentacion wins
Token :  class
Token Frequency :  [1 9]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  9
Documentacion wins
Token :  hand
Token Frequency :  [1 1]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  1
Token :  time
Token Frequency :  [3 1]
Token Repetition 2
Canciones Token count  3
Documentacion Token count  1
Canciones wins
Token :  times
Token Frequency :  [1 1]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  1
Token :  true
Token Frequency :  [2 2]
Token Repetition 2
Canciones Token count  2
Documentacion Token count  2
Token :  try
Token Frequency :  [1 4]
Token Repetition 2
Canciones Token count  1
Documentacion Token count  4
Documentacion wins
Token :  way
Token Frequency :  [3 1]
Token Repetition 2
Canciones Token count  3
Documentacion Token count  1
Canciones wins
Total terms  8


In [30]:
cancionesc = []
for string in canciones_df:
  print(string)
  cancionesc.append(' '.join(i for i in string.split() if i not in cancionesdelete))

i've got sunshine on a cloudy day with my girl
i've got a sweeter song than the birds in the trees
give you some time to prove that i can trust you again
and catch the tears whenever you cry
and in my dreams i've kissed your lips a thousand times
i'm out of my head hopelessly devoted to you
and now you know i can´t smile without you
i go out every night and sleep all day
just to let you know that i'm still around
well life would still go on believe me
oh then turn around and make up
you're all i've ever wanted and my arms are open wide
if she said that's the way it ought to be
though life would still go on believe me
now my baby's dancin' but she's dancin' with another man
and that you'd be reminded that for me, it isn't over
i'll think of you each step of the way
somewhere in her smile she knows
it's been so lonely without you here
you're my end and my beginning
is there really no chance to start once again?
they say true love's the greatest weapon
when it's cold outside i've got the 

In [31]:
documentacionc = []
for string in documentacion_df:
  print(string)
  cancionesc.append(' '.join(i for i in string.split() if i not in documentaciondelete))

a namespace is a container that holds classes structs and enums.
the using directive allows you to use types from a namespace without fully qualifying their names.
a static class cannot be instantiated and contains only static members.
a constructor initializes a new instance of a class.
the override keyword is used to provide a new implementation for a method inherited from a base class.
an interface defines a contract that classes can implement.
a struct is a value type that can contain data members and methods.
an abstract class cannot be instantiated and must be inherited by other classes.
the virtual keyword allows a method to be overridden in a derived class.
an event allows a class to provide notifications to other classes or objects.
a delegate is a type that represents references to methods with a specific parameter list and return type.
the params keyword allows a method to accept a variable number of arguments.
the async keyword defines a method that can run asynchronously.


In [32]:
# Join individual documents to calculate cosine similarity after cleansing
# Reduction of the cosine similarity
canciones = ' '.join(cancionesc)
documentacion = ' '.join(documentacionc)
documents = [canciones, documentacion]
count_vectorizer = CountVectorizer(stop_words="english")
#count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vectorizer.get_feature_names_out(),
   index=["canciones", "documentacion"],
)
print(df)
print(cosine_similarity(df, df))

               abstract  accept  allows  arguments  arms  assigned  \
canciones             1       2       6          2     1         1   
documentacion         0       0       0          0     0         0   

               assignment  async  asynchronously  attempts  ...  way  weapon  \
canciones               1      2               1         1  ...    3       1   
documentacion           0      0               0         0  ...    0       0   

               weeping  wide  willing  willow  wise  word  wrapped  write  
canciones            1     1        1       1     1     1        1      1  
documentacion        0     0        0       0     0     0        0      0  

[2 rows x 287 columns]
[[1. 0.]
 [0. 0.]]


In [33]:
fullTrainingDatac = cancionesc + documentacionc
cancionesvaluesc = [14] * len(cancionesc)
documentacionvaluesc = [15] * len(documentacionc)
fullDataClassificationc = cancionesvaluesc + documentacionvaluesc

count_vect = CountVectorizer(stop_words='english')
X_products_countsc = count_vect.fit_transform(fullTrainingDatac)
X_products_countsc.shape

(98, 287)

In [36]:
fullTrainingData = canciones_df + documentacion_df

In [37]:
cancionesvalues = [14] * len(canciones_df)
documentacionvalues = [15] * len(documentacion_df)
fullDataClassification = cancionesvalues + documentacionvalues

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
X_products_counts = count_vect.fit_transform(fullTrainingData)
X_products_counts.shape

(98, 287)

These are the term matrices

In [39]:
doc_term_matrix = X_products_counts.todense()
df = pd.DataFrame(
   doc_term_matrix,
   columns=count_vect.get_feature_names_out()
)
print(df)
print(cosine_similarity(df, df))

    abstract  accept  allows  arguments  arms  assigned  assignment  async  \
0          0       0       0          0     0         0           0      0   
1          0       0       0          0     0         0           0      0   
2          0       0       0          0     0         0           0      0   
3          0       0       0          0     0         0           0      0   
4          0       0       0          0     0         0           0      0   
..       ...     ...     ...        ...   ...       ...         ...    ...   
93         0       0       0          0     0         0           0      0   
94         0       0       0          0     0         0           1      0   
95         0       0       0          0     0         0           0      0   
96         0       0       0          0     0         0           0      0   
97         0       0       0          0     0         0           0      0   

    asynchronously  attempts  ...  way  weapon  weeping  wide  

Create Inverse Matrix

CountVectorizer team 2

In [40]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_products_counts)
X_train_tfidf.shape

(98, 287)

In [ ]:
classlookup = None
filename = 'sample_data/test_data_ml_classifier.csv'
with open(filename, newline='') as f:
                reader = csv.reader(f)
                try:
                    classlookup = defaultdict(list)
                    for key, val in reader:
                        classlookup[key] = val
                except csv.Error as e:
                    sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))

In [ ]:
test_set_golden = list(classlookup.values())

In [ ]:
test_set_golden = test_set_golden[1:]

In [ ]:
test_set_golden = [int(i) for i in test_set_golden]

In [ ]:
docs_test = list(classlookup.keys())[1:]

In [ ]:
MultinomialNB

In [ ]:
CountVectorizer

In [ ]:
TfidfTransformer

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [ ]:
text_clf.fit(fullTrainingData, fullDataClassification)